In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
!pip -q install pandas-ta==0.3.14b0 --force-reinstall
import numpy as np
# from numpy import NaN as npNaN
npNaN = np.nan
!sed -i 's/from numpy import NaN as npNaN/import numpy as np\nnpNaN = np.nan/' /usr/local/lib/python3.11/dist-packages/pandas_ta/momentum/squeeze_pro.py
import pandas_ta as ta
from datetime import date
from datetime import datetime, date
!pip -q install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git
from tvDatafeed import TvDatafeed, Interval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you ha

In [10]:
tv = TvDatafeed(username="adam799", password="cix87879@jioso.com")
try:
  os.mkdir('/content/EGX_Datasets')
except:
  pass

def get_data_tradingView(name,n_bars=100000):
  df = tv.get_hist(symbol="EGX:"+str(name), exchange="EGX", interval=Interval.in_daily, n_bars=100000)

  if df is None:
    print(f"Error: Could not retrieve data for symbol EGX:{name}")
    return None

  try:
    df.drop(['symbol'], axis=1, inplace=True)
  except:
    print('Nothing to be droped')

  df.index = pd.to_datetime(df.index)#.date
  df.rename(columns=lambda x: x.capitalize(), inplace=True)

  df['RSI'] = ta.rsi(df['Close'])
  df.ta.macd(append=True)
  df['SMA_20'] = df.ta.sma(length=20)
  df['SMA_50'] = df.ta.sma(length=50)
  try:df['SMA_200'] = df.ta.sma(length=200)
  except:pass
  df.ta.adx(append=True)
  df.ta.stoch(append=True)
  df.ta.cci(append=True)
  df.ta.bbands(append=True)
  df['OBV'] = ta.obv(close=df['Close'], volume=df['Volume'])
  # احسب Ichimoku
  ichimoku_df, _ = ta.ichimoku(df['High'], df['Low'], df['Close'])
  # دمج أعمدة Ichimoku مع df
  df = pd.concat([df, ichimoku_df], axis=1)


  # حساب المتوسط المتحرك للحجم على 10 أيام
  df['Volume_MA'] = df['Volume'].rolling(window=10).mean()

  # حساب الـ Volume Trend اليومي
  df['Volume_Trend'] = (df['Volume'] - df['Volume_MA']) / df['Volume_MA'] * 100

  # التأكد من إن مفيش قيم Infinity أو NaN
  df['Volume_Trend'] = df['Volume_Trend'].replace([np.inf, -np.inf], 0)
  df['Volume_Trend'] = df['Volume_Trend'].fillna(0)

  # حذف العمود المساعد (Volume_MA) لو مش محتاجه
  df = df.drop(columns=['Volume_MA'])

  df['Price_Volume_Corr'] = df['Close'].rolling(window=7).corr(df['Volume']).fillna(0)

  df.rename(columns={
    'MACD_12_26_9': 'MACD',
    'MACDh_12_26_9': 'MACD_Hist',
    'MACDs_12_26_9': 'MACD_Signal',
    'ADX_14': 'ADX',
    'DMP_14': 'ADX_Plus_DI',
    'DMN_14': 'ADX_Minus_DI',
    'STOCHk_14_3_3': 'Stoch_K',
    'STOCHd_14_3_3': 'Stoch_D',
    'CCI_14_0.015': 'CCI',
    'BBL_5_2.0': 'BB_Lower',
    'BBM_5_2.0': 'BB_Middle',
    'BBU_5_2.0': 'BB_Upper',
    'BBP_5_2.0': 'BB_Percentage'
    }, inplace=True)

  df.dropna(inplace=True)

  save_path = ('/content/EGX_Datasets/'+name+'_data.csv')
  df.to_csv(save_path)
  return save_path

ERROR:tvDatafeed.main:error while signin


In [3]:
def create_weekly_text_description(row, df):
    score = 0
    row_index = row.name
    start_index = max(0, row_index - 6)
    last_7_days = df.iloc[start_index:row_index + 1]

    # Trend Direction
    higher_high_count = last_7_days['Higher_High_7d'].sum()
    higher_low_count = last_7_days['Higher_Low_7d'].sum()
    if higher_high_count >= 5 and higher_low_count >= 5:
        trend_direction = 'The stock shows Higher Highs and Higher Lows in 5+ of the last 7 days, indicating a **very strong uptrend**.'
        score += 4  # إيجابي جدًا
    elif higher_high_count == 4 and higher_low_count == 4:
        trend_direction = 'The stock shows Higher Highs and Higher Lows in 4 of the last 7 days, indicating an **uptrend**.'
        score += 2  # إيجابي
    elif higher_high_count <= 2 and higher_low_count <= 2:
        trend_direction = 'The stock shows Lower Highs and Lower Lows in 5+ of the last 7 days, indicating a **very strong downtrend**.'
        score -= 4  # سلبي جدًا
    elif higher_high_count == 3 and higher_low_count == 3:
        trend_direction = 'The stock shows Lower Highs and Lower Lows in 4 of the last 7 days, indicating a **downtrend**.'
        score -= 2  # سلبي
    else:
        trend_direction = 'The stock shows a mixed trend over the last 7 days with no clear Higher Highs or Higher Lows pattern, suggesting a **sideways movement**.'
        # محايد: score لا يتغير

    # Volume Analysis
    volume_analysis = ''
    if row['Relative_Volume_7d'] > 2:
        volume_analysis = 'Trading volume is very high (%.2f times the average), suggesting very strong momentum and a **very positive** movement.' % row['Relative_Volume_7d']
        score += 4  # إيجابي جدًا
    elif 1.5 < row['Relative_Volume_7d'] <= 2:
        volume_analysis = 'Trading volume is high (%.2f times the average), suggesting strong momentum and a **positive** movement.' % row['Relative_Volume_7d']
        score += 2  # إيجابي
    elif row['Relative_Volume_7d'] < 0.3:
        volume_analysis = 'Trading volume is very low (%.2f times the average), indicating very weak momentum and a **very negative** movement.' % row['Relative_Volume_7d']
        score -= 4  # سلبي جدًا
    elif 0.3 <= row['Relative_Volume_7d'] < 0.5:
        volume_analysis = 'Trading volume is low (%.2f times the average), indicating weak momentum and a **negative** movement.' % row['Relative_Volume_7d']
        score -= 2  # سلبي
    else:
        volume_analysis = 'Trading volume is normal (%.2f times the average), suggesting a **neutral** movement.' % row['Relative_Volume_7d']
        # محايد: score لا يتغير

    # MACD Analysis
    macd_analysis = ''
    macd_trend = 'bullish' if row['MACD'] > row['MACD_Signal'] else 'bearish'
    if macd_trend == 'bullish':
        if row['MACD_Hist'] > 0.5:
            macd_analysis = 'MACD indicates a **very strong bullish** trend with a histogram of %.2f.' % row['MACD_Hist']
            score += 2  # إيجابي جدًا
        else:
            macd_analysis = 'MACD indicates a **bullish** trend with a histogram of %.2f.' % row['MACD_Hist']
            score += 1  # إيجابي
    else:
        if row['MACD_Hist'] < -0.5:
            macd_analysis = 'MACD indicates a **very strong bearish** trend with a histogram of %.2f.' % row['MACD_Hist']
            score -= 2  # سلبي جدًا
        else:
            macd_analysis = 'MACD indicates a **bearish** trend with a histogram of %.2f.' % row['MACD_Hist']
            score -= 1  # سلبي

    # RSI Analysis
    rsi_analysis = ''
    if row['RSI'] > 80:
        rsi_analysis = 'RSI at %.2f indicates **extremely overbought** conditions, suggesting a **strong reversal risk**.' % row['RSI']
        score -= 2  # سلبي جدًا
    elif 70 < row['RSI'] <= 80:
        rsi_analysis = 'RSI at %.2f indicates **overbought** conditions, suggesting a potential reversal.' % row['RSI']
        score -= 1  # سلبي
    elif row['RSI'] < 20:
        rsi_analysis = 'RSI at %.2f indicates **extremely oversold** conditions, suggesting a **strong reversal opportunity**.' % row['RSI']
        score += 2  # إيجابي جدًا
    elif 20 <= row['RSI'] < 30:
        rsi_analysis = 'RSI at %.2f indicates **oversold** conditions, suggesting a potential reversal.' % row['RSI']
        score += 1  # إيجابي
    else:
        rsi_analysis = 'RSI at %.2f indicates **neutral** conditions.' % row['RSI']
        # محايد: score لا يتغير

    # ADX Analysis
    adx_analysis = ''
    if row['ADX'] > 40:
        adx_analysis = 'ADX at %.2f indicates a **very strong trend**, strongly supporting the current movement.' % row['ADX']
        score += 2  # إيجابي جدًا
    elif 25 < row['ADX'] <= 40:
        adx_analysis = 'ADX at %.2f indicates a **strong trend**, supporting the current movement.' % row['ADX']
        score += 1  # إيجابي
    elif row['ADX'] < 10:
        adx_analysis = 'ADX at %.2f indicates a **very weak trend**, suggesting a **strong sideways movement**.' % row['ADX']
        score -= 2  # سلبي جدًا
    elif 10 <= row['ADX'] < 15:
        adx_analysis = 'ADX at %.2f indicates a **weak trend**, suggesting a **sideways movement**.' % row['ADX']
        score -= 1  # سلبي
    else:
        adx_analysis = 'ADX at %.2f indicates a **neutral trend**.' % row['ADX']
        # محايد: score لا يتغير

    # SMA Analysis
    sma_analysis = ''
    price_to_sma_ratio = (row['Close'] - row['SMA']) / row['SMA'] * 100  # النسبة المئوية
    if price_to_sma_ratio > 5:
        sma_analysis = 'The price is significantly above the SMA (%.2f), indicating a **very positive** trend.' % row['SMA']
        score += 2  # إيجابي جدًا
    elif 0 < price_to_sma_ratio <= 5:
        sma_analysis = 'The price is above the SMA (%.2f), indicating a **positive** trend.' % row['SMA']
        score += 1  # إيجابي
    elif price_to_sma_ratio < -5:
        sma_analysis = 'The price is significantly below the SMA (%.2f), indicating a **very negative** trend.' % row['SMA']
        score -= 2  # سلبي جدًا
    elif -5 <= price_to_sma_ratio < 0:
        sma_analysis = 'The price is below the SMA (%.2f), indicating a **negative** trend.' % row['SMA']
        score -= 1  # سلبي
    else:
        sma_analysis = 'The price is at the SMA (%.2f), indicating a **neutral** trend.' % row['SMA']
        # محايد: score لا يتغير

    # Stochastic Analysis
    stochastic_analysis = ''
    if row['Stoch_K'] > 90:
        stochastic_analysis = 'Stochastic K at %.2f indicates **extremely overbought** conditions, suggesting a **strong reversal risk**.' % row['Stoch_K']
        score -= 2  # سلبي جدًا
    elif 80 < row['Stoch_K'] <= 90:
        stochastic_analysis = 'Stochastic K at %.2f indicates **overbought** conditions, suggesting a potential reversal.' % row['Stoch_K']
        score -= 1  # سلبي
    elif row['Stoch_K'] < 10:
        stochastic_analysis = 'Stochastic K at %.2f indicates **extremely oversold** conditions, suggesting a **strong reversal opportunity**.' % row['Stoch_K']
        score += 2  # إيجابي جدًا
    elif 10 <= row['Stoch_K'] < 20:
        stochastic_analysis = 'Stochastic K at %.2f indicates **oversold** conditions, suggesting a potential reversal.' % row['Stoch_K']
        score += 1  # إيجابي
    else:
        stochastic_analysis = 'Stochastic K at %.2f indicates **neutral** momentum.' % row['Stoch_K']
        # محايد: score لا يتغير

    text = f"Over the past 7 days, the stock movement can be described as follows: {trend_direction} {volume_analysis} {macd_analysis} {rsi_analysis} {adx_analysis} {sma_analysis} {stochastic_analysis} (Score: {score})"
    return text, score

#In Arabic
def create_weekly_text_description_ar(row, df):
    score = 0
    row_index = row.name
    start_index = max(0, row_index - 6)
    last_7_days = df.iloc[start_index:row_index + 1]

    # تعريف متغيرات للكلمات الإنجليزية
    macd = 'الماك'
    rsi = 'ار اي اس'
    adx = 'ايه دي اكس'
    sma = 'اس ام ايه'
    stochastic = 'Stochastic'
    stochastic_k = 'الاوستاتستك'
    obv = 'اوه بي في'
    fibonacci = 'فابوناشي'
    ichimoku = 'اشيموكي'
    cci = 'سي سي اي'

    # تحليل الحجم
    volume_analysis = ''
    if row['Relative_Volume_7d'] > 2:
        volume_analysis = f'حجم التداول مرتفع جدا {row["Relative_Volume_7d"]:.2f} مرة من المتوسط مما يشير إلى نشاط سوقي قوي جدا.'
        score += 4
    elif 1.5 < row['Relative_Volume_7d'] <= 2:
        volume_analysis = f'حجم التداول مرتفع {row["Relative_Volume_7d"]:.2f} مرة من المتوسط مما يشير إلى نشاط سوقي قوي.'
        score += 2
    elif row['Relative_Volume_7d'] < 0.3:
        volume_analysis = f'حجم التداول منخفض جدا {row["Relative_Volume_7d"]:.2f} مرة من المتوسط مما يشير إلى اهتمام ضعيف جدا من المستثمرين.'
        score -= 4
    elif 0.3 <= row['Relative_Volume_7d'] < 0.5:
        volume_analysis = f'حجم التداول منخفض {row["Relative_Volume_7d"]:.2f} مرة من المتوسط مما يشير إلى اهتمام ضعيف من المستثمرين.'
        score -= 2
    else:
        volume_analysis = f'حجم التداول عادي {row["Relative_Volume_7d"]:.2f} مرة من المتوسط مما يشير إلى نشاط مستقر.'

    # تحليل MACD
    macd_analysis = ''
    macd_trend = 'صاعد' if row['MACD'] > row['MACD_Signal'] else 'هابط'
    macd_strength = row["MACD_Hist"]
    adx_score = row["ADX"]

    if row['ADX'] > 40:
        adx_trend_strength = 'قوي جدًا'
    elif 25 < row['ADX'] <= 40:
        adx_trend_strength = 'قوي'
    elif row['ADX'] < 10:
        adx_trend_strength = 'ضعيف جدًا'
    elif 10 <= row['ADX'] < 15:
        adx_trend_strength = 'ضعيف'
    else:
        adx_trend_strength = 'محايد'

    if macd_trend == 'صاعد':
        if macd_strength > 0.5:
            if adx_score >= 1:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم إيجابي قوي جدًا (هيستوغرام = {macd_strength:.2f})، وهذا مدعوم باتجاه {adx_trend_strength}.'
                score += 2
            else:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم إيجابي قوي (هيستوغرام = {macd_strength:.2f})، ولكن الاتجاه {adx_trend_strength} مما قد يشير إلى زخم **مضلل**.'
                score += 1
        else:
            if adx_score >= 1:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم إيجابي معتدل (هيستوغرام = {macd_strength:.2f})، مدعوم باتجاه {adx_trend_strength}.'
                score += 1
            else:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم إيجابي ضعيف (هيستوغرام = {macd_strength:.2f})، والاتجاه {adx_trend_strength} مما قد يشير إلى إشارات **غير مؤكدة**.'
                score += 0
    else:
        if macd_strength < -0.5:
            if adx_score >= 1:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم سلبي قوي جدًا (هيستوغرام = {macd_strength:.2f})، وهذا مدعوم باتجاه {adx_trend_strength}.'
                score -= 2
            else:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم سلبي قوي (هيستوغرام = {macd_strength:.2f})، ولكن الاتجاه {adx_trend_strength} مما قد يشير إلى زخم **مضلل**.'
                score -= 1
        else:
            if adx_score >= 1:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم سلبي معتدل (هيستوغرام = {macd_strength:.2f})، مدعوم باتجاه {adx_trend_strength}.'
                score -= 1
            else:
                macd_analysis = f'مؤشر {macd} يشير إلى زخم سلبي ضعيف (هيستوغرام = {macd_strength:.2f})، والاتجاه {adx_trend_strength} مما قد يشير إلى إشارات **غير مؤكدة**.'
                score -= 0

    # تحليل RSI
    rsi_analysis = ''
    if row['RSI'] > 80:
        rsi_analysis = f'مؤشر {rsi} عند {row["RSI"]:.2f} يشير إلى حالة تشبع شرائي قوي جدا مما يعني مخاطر انعكاس قوية.'
        score -= 2
    elif 70 < row['RSI'] <= 80:
        rsi_analysis = f'مؤشر {rsi} عند {row["RSI"]:.2f} يشير إلى حالة تشبع شرائي مما قد يعني تباطؤ أو انعكاس.'
        score -= 1
    elif row['RSI'] < 20:
        rsi_analysis = f'مؤشر {rsi} عند {row["RSI"]:.2f} يشير إلى حالة تشبع بيعي قوي جدا مما يوحي بفرصة صعود قوية.'
        score += 2
    elif 20 <= row['RSI'] < 30:
        rsi_analysis = f'مؤشر {rsi} عند {row["RSI"]:.2f} يشير إلى حالة تشبع بيعي مما يوحي بإمكانية تصحيح صعودي.'
        score += 1
    else:
        rsi_analysis = f'مؤشر {rsi} عند {row["RSI"]:.2f} يعكس حالة عادية في السوق.'

    # تحليل Stochastic
    stochastic_analysis = ''
    if row['Stoch_K'] > 90:
        stochastic_analysis = f'مؤشر {stochastic_k} عند {row["Stoch_K"]:.2f} يشير إلى مستويات تشبع شرائي قوي جدا مما يوحي بمخاطر انعكاس قوية.'
        score -= 2
    elif 80 < row['Stoch_K'] <= 90:
        stochastic_analysis = f'مؤشر {stochastic_k} عند {row["Stoch_K"]:.2f} يشير إلى مستويات تشبع شرائي.'
        score -= 1
    elif row['Stoch_K'] < 10:
        stochastic_analysis = f'مؤشر {stochastic_k} عند {row["Stoch_K"]:.2f} يشير إلى مستويات تشبع بيعي قوي جدا مما يوحي بفرصة صعود قوية.'
        score += 2
    elif 10 <= row['Stoch_K'] < 20:
        stochastic_analysis = f'مؤشر {stochastic_k} عند {row["Stoch_K"]:.2f} يشير إلى مستويات تشبع بيعي.'
        score += 1
    else:
        stochastic_analysis = f'مؤشر {stochastic_k} عند {row["Stoch_K"]:.2f} يشير إلى زخم محايد.'

    # تحليل CCI
    cci_analysis_f = ''
    cci_analysis = ''
    price_action_analysis = ''

    # افترض أن لديك دالة للتحقق من نمط الشمعة
    def check_candlestick_pattern():
        # مثال على نمط شمعة (هنا مجرد مثال)
        if row['Close'] > row['Open']:  # شمعة صاعدة (مثلاً)
            return "شمعة صاعدة"
        elif row['Close'] < row['Open']:  # شمعة هابطة (مثلاً)
            return "شمعة هابطة"
        else:
            return "شمعة محايدة"

    # افترض أن لديك دالة للتحقق من مستوى الدعم/المقاومة
    def check_support_resistance(df, lookback_period=50):
        recent_data = df.tail(lookback_period)  # الحصول على آخر 50 يومًا من البيانات (أو أي فترة أخرى)
        support_level = recent_data['Low'].min()
        resistance_level = recent_data['High'].max()

        current_price = df.iloc[-1]['Close']  # السعر الحالي (آخر سعر إغلاق)

        # التحقق إذا كان السعر عند مستوى الدعم أو المقاومة
        if current_price <= support_level * 1.05:  # إذا كان السعر قريبًا من مستوى الدعم (مثلاً +/- 5%)
            return "دعم قوي"
        elif current_price >= resistance_level * 0.95:  # إذا كان السعر قريبًا من مستوى المقاومة (مثلاً +/- 5%)
            return "مقاومة قوية"
        else:
            return "لا يوجد مستوى دعم أو مقاومة واضح"

    # تحليل CCI
    if row['CCI'] > 200:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى حالة تشبع شرائي قوي جدا مما يعني مخاطر انعكاس قوية.'
    elif 100 < row['CCI'] <= 200:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى حالة تشبع شرائي مما قد يعني تباطؤ أو انعكاس.'
    elif 0 < row['CCI'] <= 100:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى زخم إيجابي بسيط.'
    elif row['CCI'] < -200:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى حالة تشبع بيعي قوي جدا مما يوحي بفرصة صعود قوية.'
    elif -200 <= row['CCI'] < -100:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى حالة تشبع بيعي مما يوحي بإمكانية تصحيح صعودي.'
    elif -100 <= row['CCI'] < 0:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى زخم سلبي بسيط.'
    else:
        cci_analysis = f'مؤشر {cci} عند {row["CCI"]:.2f} يشير إلى زخم محايد.'

    # دمج التحليل مع نمط السعر
    price_action = check_candlestick_pattern()
    support_resistance = check_support_resistance(df)

    if price_action == "شمعة صاعدة" and row['CCI'] > 100:
        price_action_analysis = f'نمط السعر: {price_action} مع {cci} إيجابي يشير إلى احتمالية استمرار الزخم الصاعد.'
        score += 1
    elif price_action == "شمعة هابطة" and row['CCI'] < -100:
        price_action_analysis = f'نمط السعر: {price_action} مع {cci} سلبي يشير إلى احتمالية استمرار الزخم الهابط.'
        score -= 1
    elif support_resistance == "مقاومة قوية" and row['CCI'] > 200:
        price_action_analysis = f'السعر عند مقاومة قوية مع {cci} في منطقة تشبع شرائي يشير إلى احتمال انعكاس هبوطي.'
        score -= 3
    elif support_resistance == "دعم قوي" and row['CCI'] < -200:
        price_action_analysis = f'السعر عند دعم قوي مع {cci} في منطقة تشبع بيعي يشير إلى احتمال انعكاس صاعد.'
        score += 3
    else:
        price_action_analysis = f'نمط السعر لا يتعارض مع تحليل {cci}.'

    # دمج التحليلين معًا
    cci_analysis_f = cci_analysis + " " + price_action_analysis

    text = f'خلال ال 7 أيام الماضية يمكن وصف حركة السهم كالتالي  {volume_analysis} {macd_analysis} {rsi_analysis} {stochastic_analysis} {cci_analysis_f} (النقاط: {score})'
    return text, score

In [4]:
def create_monthly_text_description(row, df):
    score = 0
    row_index = row.name
    start_index = max(0, row_index - 29)
    last_30_days = df.iloc[start_index:row_index + 1]

    # Trend Direction
    higher_high_count = last_30_days['Higher_High_21d'].sum()
    higher_low_count = last_30_days['Higher_Low_21d'].sum()
    if higher_high_count >= 18 and higher_low_count >= 18:
        trend_direction = 'The stock shows a **very strong positive** long-term trend with consistent Higher Highs and Higher Lows in 18+ days over the past month.'
        score += 4  # إيجابي جدًا
    elif higher_high_count >= 16 and higher_low_count >= 16:
        trend_direction = 'The stock shows a **positive** long-term trend with consistent Higher Highs and Higher Lows in 16-17 days over the past month.'
        score += 2  # إيجابي
    elif higher_high_count <= 12 and higher_low_count <= 12:
        trend_direction = 'The stock shows a **very strong negative** long-term trend with consistent Lower Highs and Lower Lows in 18+ days over the past month.'
        score -= 4  # سلبي جدًا
    elif higher_high_count <= 14 and higher_low_count <= 14:
        trend_direction = 'The stock shows a **negative** long-term trend with consistent Lower Highs and Lower Lows in 16-17 days over the past month.'
        score -= 2  # سلبي
    else:
        trend_direction = 'The stock exhibits a **neutral** long-term trend over the past month with mixed patterns.'
        # محايد: score لا يتغير

    # Volume Analysis
    volume_analysis = ''
    if row['Relative_Volume_21d'] > 2:
        volume_analysis = 'The trading volume is **very high** (%.2f times the average), indicating very strong market activity.' % row['Relative_Volume_21d']
        score += 4  # إيجابي جدًا
    elif 1.5 < row['Relative_Volume_21d'] <= 2:
        volume_analysis = 'The trading volume is **positively** higher than average (%.2f times), indicating strong market activity.' % row['Relative_Volume_21d']
        score += 2  # إيجابي
    elif row['Relative_Volume_21d'] < 0.3:
        volume_analysis = 'The trading volume is **very low** (%.2f times the average), suggesting very weak investor interest.' % row['Relative_Volume_21d']
        score -= 4  # سلبي جدًا
    elif 0.3 <= row['Relative_Volume_21d'] < 0.5:
        volume_analysis = 'The trading volume is **negatively** lower than average (%.2f times), suggesting weak investor interest.' % row['Relative_Volume_21d']
        score -= 2  # سلبي
    else:
        volume_analysis = 'The trading volume remains **normal** (%.2f times), indicating stable activity.' % row['Relative_Volume_21d']
        # محايد: score لا يتغير

    # MACD Analysis
    macd_analysis = ''
    macd_trend = 'bullish' if row['MACD'] > row['MACD_Signal'] else 'bearish'
    if macd_trend == 'bullish':
        if row['MACD_Hist'] > 0.5:
            macd_analysis = 'MACD suggests a **very strong positive** momentum with a histogram reading of %.2f.' % row['MACD_Hist']
            score += 2  # إيجابي جدًا
        else:
            macd_analysis = 'MACD suggests a **positive** momentum with a histogram reading of %.2f.' % row['MACD_Hist']
            score += 1  # إيجابي
    else:
        if row['MACD_Hist'] < -0.5:
            macd_analysis = 'MACD suggests a **very strong negative** momentum with a histogram reading of %.2f.' % row['MACD_Hist']
            score -= 2  # سلبي جدًا
        else:
            macd_analysis = 'MACD suggests a **negative** momentum with a histogram reading of %.2f.' % row['MACD_Hist']
            score -= 1  # سلبي

    # RSI Analysis
    rsi_analysis = ''
    if row['RSI'] > 80:
        rsi_analysis = 'RSI at %.2f indicates an **extremely overbought** condition, signaling a strong risk of reversal.' % row['RSI']
        score -= 2  # سلبي جدًا
    elif 70 < row['RSI'] <= 80:
        rsi_analysis = 'RSI at %.2f indicates an **overbought** condition, signaling a possible slowdown or reversal.' % row['RSI']
        score -= 1  # سلبي
    elif row['RSI'] < 20:
        rsi_analysis = 'RSI at %.2f indicates an **extremely oversold** condition, suggesting a strong potential for upward correction.' % row['RSI']
        score += 2  # إيجابي جدًا
    elif 20 <= row['RSI'] < 30:
        rsi_analysis = 'RSI at %.2f indicates an **oversold** condition, suggesting potential for upward correction.' % row['RSI']
        score += 1  # إيجابي
    else:
        rsi_analysis = 'RSI at %.2f reflects a **normal** market condition.' % row['RSI']
        # محايد: score لا يتغير

    # ADX Analysis
    adx_analysis = ''
    if row['ADX'] > 40:
        adx_analysis = 'ADX at %.2f highlights a **very strong** trend.' % row['ADX']
        score += 2  # إيجابي جدًا
    elif 25 < row['ADX'] <= 40:
        adx_analysis = 'ADX at %.2f highlights a **strong** trend.' % row['ADX']
        score += 1  # إيجابي
    elif row['ADX'] < 10:
        adx_analysis = 'ADX at %.2f suggests a **very weak** trend or **strong sideways movement**.' % row['ADX']
        score -= 2  # سلبي جدًا
    elif 10 <= row['ADX'] < 15:
        adx_analysis = 'ADX at %.2f suggests a **weak** trend or **sideways movement**.' % row['ADX']
        score -= 1  # سلبي
    else:
        adx_analysis = 'ADX at %.2f indicates a **neutral** trend.' % row['ADX']
        # محايد: score لا يتغير

    # SMA Analysis
    sma_analysis = ''
    price_to_sma_ratio = (row['Close'] - row['SMA']) / row['SMA'] * 100  # النسبة المئوية
    if price_to_sma_ratio > 5:
        sma_analysis = 'The price is **significantly above** the SMA (%.2f), supporting a **very positive** market view.' % row['SMA']
        score += 2  # إيجابي جدًا
    elif 0 < price_to_sma_ratio <= 5:
        sma_analysis = 'The price is **above** the SMA (%.2f), supporting a **positive** market view.' % row['SMA']
        score += 1  # إيجابي
    elif price_to_sma_ratio < -5:
        sma_analysis = 'The price is **significantly below** the SMA (%.2f), supporting a **very negative** market view.' % row['SMA']
        score -= 2  # سلبي جدًا
    elif -5 <= price_to_sma_ratio < 0:
        sma_analysis = 'The price is **below** the SMA (%.2f), supporting a **negative** market view.' % row['SMA']
        score -= 1  # سلبي
    else:
        sma_analysis = 'The price is **at** the SMA (%.2f), indicating a **neutral** trend.' % row['SMA']
        # محايد: score لا يتغير

    # Stochastic Analysis
    stochastic_analysis = ''
    if row['Stoch_K'] > 90:
        stochastic_analysis = 'Stochastic K at %.2f signals **extremely overbought** levels, suggesting a strong risk of reversal.' % row['Stoch_K']
        score -= 2  # سلبي جدًا
    elif 80 < row['Stoch_K'] <= 90:
        stochastic_analysis = 'Stochastic K at %.2f signals **overbought** levels.' % row['Stoch_K']
        score -= 1  # سلبي
    elif row['Stoch_K'] < 10:
        stochastic_analysis = 'Stochastic K at %.2f signals **extremely oversold** levels, suggesting a strong potential for upward correction.' % row['Stoch_K']
        score += 2  # إيجابي جدًا
    elif 10 <= row['Stoch_K'] < 20:
        stochastic_analysis = 'Stochastic K at %.2f signals **oversold** levels.' % row['Stoch_K']
        score += 1  # إيجابي
    else:
        stochastic_analysis = 'Stochastic K at %.2f indicates **neutral** momentum.' % row['Stoch_K']
        # محايد: score لا يتغير

    text = f"Over the past 30 days, the stock performance can be summarized as follows: {trend_direction} {volume_analysis} {macd_analysis} {rsi_analysis} {adx_analysis} {sma_analysis} {stochastic_analysis} (Score: {score})"
    return text, score

#In Arabic
def create_monthly_text_description_ar(row, df):
    score = 0
    row_index = row.name
    start_index = max(0, row_index - 29)
    last_30_days = df.iloc[start_index:row_index + 1]

    # تعريف متغيرات للكلمات الإنجليزية
    # adx = 'ADX'
    # sma = 'SMA'
    # obv = 'OBV'
    # fibonacci = 'Fibonacci Retracement'
    # ichimoku = 'Ichimoku Cloud'
    adx = 'ايه دي اكس'
    sma = 'اس ام ايه'
    sma20 = 'اس ام ايه 20'
    sma50 = 'اس ام ايه 50'
    sma200 = 'اس ام ايه 200'
    obv = 'اوه بي في'
    fibonacci = 'فابوناشي'
    ichimoku = 'الإيشيموكو'

    # تحليل الاتجاه
    # دمج التحليل مع تصحيح السعر وحجم التداول
    higher_high_count = last_30_days['Higher_High_21d'].sum()
    higher_low_count = last_30_days['Higher_Low_21d'].sum()
    price_volume_corr = row['Price_Volume_Corr']

    # تحليل الاتجاه بناءً على Higher Highs / Higher Lows
    if higher_high_count >= 18 and higher_low_count >= 18:
        trend_direction = 'السهم يظهر اتجاها إيجابيا قويا جدا على المدى الطويل مع ارتفاعات وانخفاضات أعلى باستمرار في 18 يوما أو أكثر خلال الشهر الماضي.'
    elif higher_high_count >= 16 and higher_low_count >= 16:
        trend_direction = 'السهم يظهر اتجاها إيجابيا على المدى الطويل مع ارتفاعات وانخفاضات أعلى باستمرار في 16-17 يوما خلال الشهر الماضي.'
    elif higher_high_count <= 12 and higher_low_count <= 12:
        trend_direction = 'السهم يظهر اتجاها سلبيا قويا جدا على المدى الطويل مع ارتفاعات وانخفاضات أقل باستمرار في 18 يوما أو أكثر خلال الشهر الماضي.'
    elif higher_high_count <= 14 and higher_low_count <= 14:
        trend_direction = 'السهم يظهر اتجاها سلبيا على المدى الطويل مع ارتفاعات وانخفاضات أقل باستمرار في 16-17 يوما خلال الشهر الماضي.'
    else:
        trend_direction = 'السهم يظهر اتجاها محايدا على المدى الطويل خلال الشهر الماضي مع أنماط مختلطة.'

    # تحليل ارتباط السعر بحجم التداول (Price-Volume Correlation)
    if price_volume_corr > 0.5:
        pv_analysis = 'الاتجاه قوي جداً مع حجم تداول مرتفع، مما يدعم الاتجاه الحالي بشكل كبير.'
        score += 4
    elif 0.2 < price_volume_corr <= 0.5:
        pv_analysis = 'الاتجاه قوي مع دعم من حجم تداول مرتفع إلى متوسط.'
        score += 2
    elif -0.5 <= price_volume_corr <= 0.2:
        pv_analysis = 'الاتجاه ضعيف أو محايد، وحجم التداول غير داعم له بشكل قوي.'
        score -= 2
    else:
        pv_analysis = 'الاتجاه ضعيف جداً مع حجم تداول منخفض، مما قد يعني ضعف في الاتجاه الحالي.'
        score -= 4

    # دمج تحليل الاتجاه مع تحليل حجم التداول
    trend_direction = f"{trend_direction} {pv_analysis}"


    # تحليل ADX
    adx_analysis = ''
    if row['ADX'] > 40:
        adx_analysis = f'مؤشر {adx} عند {row["ADX"]:.2f} يبرز اتجاها قويا جدا.'
        score += 2
    elif 25 < row['ADX'] <= 40:
        adx_analysis = f'مؤشر {adx} عند {row["ADX"]:.2f} يبرز اتجاها قويا.'
        score += 1
    elif row['ADX'] < 10:
        adx_analysis = f'مؤشر {adx} عند {row["ADX"]:.2f} يشير إلى اتجاه ضعيف جدا أو حركة جانبية قوية.'
        score -= 2
    elif 10 <= row['ADX'] < 15:
        adx_analysis = f'مؤشر {adx} عند {row["ADX"]:.2f} يشير إلى اتجاه ضعيف أو حركة جانبية.'
        score -= 1
    else:
        adx_analysis = f'مؤشر {adx} عند {row["ADX"]:.2f} يشير إلى اتجاه محايد.'

    # تحليل SMA
    # تحليل SMA لمختلف الفترات
    sma_analysis = ''
    sma_20 = row['SMA_20']
    sma_50 = row['SMA_50']
    sma_200 = row['SMA_200']

    # حساب نسبة السعر إلى المتوسط المتحرك SMA
    price_to_sma_20_ratio = (row['Close'] - sma_20) / sma_20 * 100
    price_to_sma_50_ratio = (row['Close'] - sma_50) / sma_50 * 100
    price_to_sma_200_ratio = (row['Close'] - sma_200) / sma_200 * 100

    # تحليل التقاطعات بين المتوسطات المتحركة
    if sma_20 > sma_50 and sma_50 > sma_200:
        sma_analysis = f'الاتجاه الصعودي قوي حيث أن {sma20} أعلى من {sma50} و{sma50} أعلى من {sma200}.'
    elif sma_20 < sma_50 and sma_50 < sma_200:
        sma_analysis = f'الاتجاه الهبوطي قوي حيث أن {sma20} أقل من {sma50} و{sma50} أقل من {sma_200}.'
    elif sma_20 > sma_50 and sma_50 < sma_200:
        sma_analysis = f'السوق يشهد تقاطع صعودي قصير الأمد بين {sma20} و{sma50}، لكن {sma50} أقل من {sma_200} مما يشير إلى ضعف في الاتجاه.'
    elif sma_20 < sma_50 and sma_50 > sma_200:
        sma_analysis = f'السوق يشهد تقاطع هبوطي قصير الأمد بين {sma20} و{sma50}، لكن {sma50} أعلى من {sma_200} مما قد يشير إلى احتمال تحول في الاتجاه.'
    else:
        sma_analysis = f'لا يوجد اتجاه واضح بناءً على تقاطعات المتوسطات المتحركة في الوقت الحالي.'

    # إضافة التحليل مع معدل السعر إلى المتوسطات المختلفة
    if price_to_sma_20_ratio > 5:
        sma_analysis += f' السعر أعلى بكثير من {sma20} {sma_20:.2f} مما يدعم رؤية سوقية إيجابية جدًا.'
        score += 2
    elif 0 < price_to_sma_20_ratio <= 5:
        sma_analysis += f' السعر أعلى من {sma20} {sma_20:.2f} مما يدعم رؤية سوقية إيجابية.'
        score += 1
    elif price_to_sma_20_ratio < -5:
        sma_analysis += f' السعر أقل بكثير من {sma_20} {sma_20:.2f} مما يدعم رؤية سوقية سلبية جدًا.'
        score -= 2
    elif -5 <= price_to_sma_20_ratio < 0:
        sma_analysis += f' السعر أقل من {sma_20} {sma_20:.2f} مما يدعم رؤية سوقية سلبية.'
        score -= 1
    else:
        sma_analysis += f' السعر عند {sma_200} {sma_20:.2f} مما يشير إلى اتجاه محايد.'

    # يمكن إضافة التحليل لبقية المتوسطات المتحركة إذا لزم الأمر، على نفس النمط.


    # تحليل OBV
    obv_analysis = ''
    prev_obv = df['OBV'].iloc[row_index - 1] if row_index > 0 else row['OBV']
    prev_price = df['Close'].iloc[row_index - 1] if row_index > 0 else row['Close']
    obv_change = (row['OBV'] - prev_obv) / prev_obv if prev_obv != 0 else 0
    price_change = (row['Close'] - prev_price) / prev_price if prev_price != 0 else 0

    if obv_change > 0.05 and price_change > 0.05:
        obv_analysis = f'مؤشر {obv} يظهر زيادة قوية جدا عند {row["OBV"]:.2f} مع ارتفاع السعر مما يدعم الاتجاه الصاعد بقوة.'
        score += 3
    elif obv_change > 0.02 and price_change > 0.02:
        obv_analysis = f'مؤشر {obv} يظهر زيادة متوسطة عند {row["OBV"]:.2f} مع ارتفاع السعر مما يدعم الاتجاه الصاعد.'
        score += 2
    elif obv_change > 0 and price_change > 0:
        obv_analysis = f'مؤشر {obv} يظهر زيادة طفيفة عند {row["OBV"]:.2f} مع ارتفاع السعر مما يشير إلى دعم بسيط للاتجاه الصاعد.'
        score += 1
    elif obv_change < -0.05 and price_change < -0.05:
        obv_analysis = f'مؤشر {obv} يظهر انخفاضا قويا جدا عند {row["OBV"]:.2f} مع انخفاض السعر مما يدعم الاتجاه الهابط بقوة.'
        score -= 3
    elif obv_change < -0.02 and price_change < -0.02:
        obv_analysis = f'مؤشر {obv} يظهر انخفاضا متوسطا عند {row["OBV"]:.2f} مع انخفاض السعر مما يدعم الاتجاه الهابط.'
        score -= 2
    elif obv_change < 0 and price_change < 0:
        obv_analysis = f'مؤشر {obv} يظهر انخفاضا طفيفا عند {row["OBV"]:.2f} مع انخفاض السعر مما يشير إلى ضعف بسيط في الاتجاه الصاعد.'
        score -= 1
    else:
        obv_analysis = f'مؤشر {obv} عند {row["OBV"]:.2f} يظهر تناقض مع حركة السعر مما قد يشير إلى انعكاس محتمل.'
        score -= 1

    # تحليل Fibonacci Retracement
    fib_analysis = ''
    high_price = last_30_days['High'].max()
    low_price = last_30_days['Low'].min()
    price_range = high_price - low_price
    fib_levels = {
        '23.6%': high_price - (price_range * 0.236),
        '38.2%': high_price - (price_range * 0.382),
        '50%': high_price - (price_range * 0.5),
        '61.8%': high_price - (price_range * 0.618),
    }
    current_price = row['Close']
    nearest_level = min(fib_levels.items(), key=lambda x: abs(x[1] - current_price))
    fib_level, fib_value = nearest_level
    price_to_fib_ratio = (current_price - fib_value) / fib_value * 100

    if price_to_fib_ratio > 5:
        fib_analysis = f'السعر عند {current_price:.2f} أعلى بكثير من مستوى {fibonacci} {fib_level} عند {fib_value:.2f} مما يشير إلى قوة صعودية كبيرة.'
        score += 4
    elif 2 < price_to_fib_ratio <= 5:
        fib_analysis = f'السعر عند {current_price:.2f} أعلى من مستوى {fibonacci} {fib_level} عند {fib_value:.2f} مما يشير إلى قوة صعودية.'
        score += 2
    elif 0 < price_to_fib_ratio <= 2:
        fib_analysis = f'السعر عند {current_price:.2f} أعلى قليلا من مستوى {fibonacci} {fib_level} عند {fib_value:.2f} مما يشير إلى دعم صعودي بسيط.'
        score += 1
    elif price_to_fib_ratio < -5:
        fib_analysis = f'السعر عند {current_price:.2f} أقل بكثير من مستوى {fibonacci} {fib_level} عند {fib_value:.2f} مما يشير إلى ضعف كبير.'
        score -= 4
    elif -5 <= price_to_fib_ratio < -2:
        fib_analysis = f'السعر عند {current_price:.2f} أقل من مستوى {fibonacci} {fib_level} عند {fib_value:.2f} مما يشير إلى ضعف.'
        score -= 2
    else:
        fib_analysis = f'السعر عند {current_price:.2f} قريب من مستوى {fibonacci} {fib_level} عند {fib_value:.2f} مما قد يشير إلى دعم أو مقاومة محتملة.'
        score -= 1

    # تحليل Ichimoku Cloud
    ichimoku_analysis = ''
    ichimoku_score = 0  # هنحسب سكور داخلي لكل مكون ثم نحدد الحالة النهائية

    # 1. السعر بالنسبة للسحابة
    if row['Close'] > row['ISA_9'] and row['Close'] > row['ISB_26']:
        price_above_cloud = True
        cloud_distance = (row['Close'] - max(row['ISA_9'], row['ISB_26'])) / max(row['ISA_9'], row['ISB_26']) * 100
        if cloud_distance > 5:
            ichimoku_score += 2  # بعيد فوق السحابة
        else:
            ichimoku_score += 1  # فوق السحابة بس قريب
    elif row['Close'] < row['ISA_9'] and row['Close'] < row['ISB_26']:
        price_above_cloud = False
        cloud_distance = (min(row['ISA_9'], row['ISB_26']) - row['Close']) / min(row['ISA_9'], row['ISB_26']) * 100
        if cloud_distance > 5:
            ichimoku_score -= 2  # بعيد تحت السحابة
        else:
            ichimoku_score -= 1  # تحت السحابة بس قريب
    else:
        price_above_cloud = None  # داخل السحابة

    # 2. تقاطع Tenkan-sen وKijun-sen
    tenkan_kijun_cross = 'صاعد' if row['ITS_9'] > row['IKS_26'] else 'هابط'
    if tenkan_kijun_cross == 'صاعد':
        ichimoku_score += 1
    else:
        ichimoku_score -= 1

    # 3. Chikou Span
    past_price = df['Close'].iloc[row_index - 26] if row_index >= 26 else row['Close']
    chikou_span_position = 'صاعد' if row['ICS_26'] > past_price else 'هابط'
    if chikou_span_position == 'صاعد':
        ichimoku_score += 1
    else:
        ichimoku_score -= 1

    # 4. اتجاه السحابة
    cloud_direction = 'صاعد' if row['ISA_9'] > row['ISB_26'] else 'هابط'
    if cloud_direction == 'صاعد':
        ichimoku_score += 1
    else:
        ichimoku_score -= 1

    # 5. سمك السحابة
    cloud_thickness = abs(row['ISA_9'] - row['ISB_26']) / min(row['ISA_9'], row['ISB_26']) * 100
    if cloud_thickness > 5:
        if cloud_direction == 'صاعد':
            ichimoku_score += 1  # سحابة سميكة صاعدة
        else:
            ichimoku_score -= 1  # سحابة سميكة هابطة

    # تحديد الحالة النهائية بناءً على الـ ichimoku_score
    if ichimoku_score >= 4:
        ichimoku_analysis = (f'مؤشر {ichimoku} يظهر إشارة صعودية قوية جدا: السعر {"فوق" if price_above_cloud else "داخل"} السحابة، '
                            f'تقاطع {tenkan_kijun_cross} بين Tenkan-sen وKijun-sen، Chikou Span {chikou_span_position}، '
                            f'واتجاه السحابة {cloud_direction} مع سمك {cloud_thickness:.2f}%.')
        score += 4
    elif 2 <= ichimoku_score < 4:
        ichimoku_analysis = (f'مؤشر {ichimoku} يظهر إشارة صعودية: السعر {"فوق" if price_above_cloud else "داخل"} السحابة، '
                            f'تقاطع {tenkan_kijun_cross} بين Tenkan-sen وKijun-sen، Chikou Span {chikou_span_position}، '
                            f'واتجاه السحابة {cloud_direction} مع سمك {cloud_thickness:.2f}%.')
        score += 2
    elif 0 < ichimoku_score < 2:
        ichimoku_analysis = (f'مؤشر {ichimoku} يظهر إشارة صعودية بسيطة: السعر {"فوق" if price_above_cloud else "داخل"} السحابة، '
                            f'تقاطع {tenkan_kijun_cross} بين Tenkan-sen وKijun-sen، Chikou Span {chikou_span_position}، '
                            f'واتجاه السحابة {cloud_direction} مع سمك {cloud_thickness:.2f}%.')
        score += 1
    elif ichimoku_score <= -4:
        ichimoku_analysis = (f'مؤشر {ichimoku} يظهر إشارة هابطة قوية جدا: السعر {"تحت" if price_above_cloud == False else "داخل"} السحابة، '
                            f'تقاطع {tenkan_kijun_cross} بين Tenkan-sen وKijun-sen، Chikou Span {chikou_span_position}، '
                            f'واتجاه السحابة {cloud_direction} مع سمك {cloud_thickness:.2f}%.')
        score -= 4
    elif -4 < ichimoku_score <= -2:
        ichimoku_analysis = (f'مؤشر {ichimoku} يظهر إشارة هابطة: السعر {"تحت" if price_above_cloud == False else "داخل"} السحابة، '
                            f'تقاطع {tenkan_kijun_cross} بين Tenkan-sen وKijun-sen، Chikou Span {chikou_span_position}، '
                            f'واتجاه السحابة {cloud_direction} مع سمك {cloud_thickness:.2f}%.')
        score -= 2
    else:
        ichimoku_analysis = (f'مؤشر {ichimoku} يظهر إشارة محايدة: السعر {"داخل" if price_above_cloud is None else "فوق" if price_above_cloud else "تحت"} السحابة، '
                            f'تقاطع {tenkan_kijun_cross} بين Tenkan-sen وKijun-sen، Chikou Span {chikou_span_position}، '
                            f'واتجاه السحابة {cloud_direction} مع سمك {cloud_thickness:.2f}%.')
        score -= 1

    text = f"خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم كالتالي: {trend_direction} {adx_analysis} {sma_analysis} {obv_analysis} {fib_analysis} {ichimoku_analysis} (النقاط: {score})"
    return text, score

In [5]:
def check_last_day_signal(df):
    past_data = df.iloc[-30:]
    # أخذ بيانات آخر يوم تداول
    row = past_data.iloc[-1]

    buy_signals = []
    sell_signals = []

    # 1. RSI
    if row['RSI'] < 30:
        buy_signals.append('RSI')
    elif row['RSI'] > 70:
        sell_signals.append('RSI')

    # 2. MACD
    if row['MACD'] > row['MACD_Signal'] and row['MACD_Hist'] > 0:
        buy_signals.append('MACD')
    elif row['MACD'] < row['MACD_Signal'] and row['MACD_Hist'] < 0:
        sell_signals.append('MACD')

    # 3. SMA
    if row['Close'] > row['SMA_20'] and past_data['Close'].iloc[-2] <= past_data['SMA_20'].iloc[-2]:
        buy_signals.append('SMA')
    elif row['Close'] < row['SMA_20'] and past_data['Close'].iloc[-2] >= past_data['SMA_20'].iloc[-2]:
        sell_signals.append('SMA')

    # 4. ADX (يحتاج تأكيد من مؤشرات أخرى)
    if row['ADX'] > 25:
        if row['Close'] > row['SMA_20']:
            buy_signals.append('ADX')
        else:
            sell_signals.append('ADX')

    # 5. Stochastic
    if row['Stoch_K'] < 20 and row['Stoch_K'] > row['Stoch_D']:
        buy_signals.append('Stochastic')
    elif row['Stoch_K'] > 80 and row['Stoch_K'] < row['Stoch_D']:
        sell_signals.append('Stochastic')

    # 6. CCI
    if row['CCI'] < -100 and past_data['CCI'].iloc[-2] < row['CCI']:
        buy_signals.append('CCI')
    elif row['CCI'] > 100 and past_data['CCI'].iloc[-2] > row['CCI']:
        sell_signals.append('CCI')

    # 7. Bollinger Bands
    if row['Close'] <= row['BB_Lower'] and past_data['Close'].iloc[-2] < row['Close']:
        buy_signals.append('Bollinger Bands')
    elif row['Close'] >= row['BB_Upper'] and past_data['Close'].iloc[-2] > row['Close']:
        sell_signals.append('Bollinger Bands')

    # 8. OBV
    prev_obv = past_data['OBV'].iloc[-2] if len(past_data) > 1 else row['OBV']
    price_change = (row['Close'] - past_data['Close'].iloc[-2]) / past_data['Close'].iloc[-2] if len(past_data) > 1 else 0
    obv_change = (row['OBV'] - prev_obv) / prev_obv if prev_obv != 0 else 0
    if obv_change > 0 and price_change > 0:
        buy_signals.append('OBV')
    elif obv_change < 0 and price_change < 0:
        sell_signals.append('OBV')

    # 9. Ichimoku Cloud (للتحليل الشهري فقط)
    row_index = df.index[-1]  # مؤشر الصف الأخير في الـ DataFrame الأصلي
    if (row['Close'] > row['ISA_9'] and row['Close'] > row['ISB_26'] and
        row['ITS_9'] > row['IKS_26'] and
        row['ISA_9'] > row['ISB_26']):
        past_price = df['Close'].iloc[df.index.get_loc(row_index) - 26] if df.index.get_loc(row_index) >= 26 else row['Close']
        if row['ICS_26'] > past_price:
            buy_signals.append('Ichimoku')
    elif (row['Close'] < row['ISA_9'] and row['Close'] < row['ISB_26'] and
          row['ITS_9'] < row['IKS_26'] and
          row['ISA_9'] < row['ISB_26']):
        past_price = df['Close'].iloc[df.index.get_loc(row_index) - 26] if df.index.get_loc(row_index) >= 26 else row['Close']
        if row['ICS_26'] < past_price:
            sell_signals.append('Ichimoku')

    signal_power = len(buy_signals) - len(sell_signals)
    # تحديد الإشارة النهائية
    if len(buy_signals) >= 3 and len(sell_signals) == 0:
        signal = f'شراء ({", ".join(buy_signals)})'
    elif len(sell_signals) >= 3 and len(buy_signals) == 0:
        signal = f'بيع ({", ".join(sell_signals)})'
    else:
        signal = 'محايد'
        signal_power = 0


    return signal,signal_power

In [6]:
def add_indicators(df):
  # Weekly indicators (7 days)
  df['Avg_Volume_7d'] = df['Volume'].shift(1).rolling(window=7).mean()
  df['Relative_Volume_7d'] = df['Volume'] / df['Avg_Volume_7d']

  df['Previous_High_7d'] = df['High'].shift(1).rolling(window=7).max()
  df['Previous_Low_7d'] = df['Low'].shift(1).rolling(window=7).min()
  df['Higher_High_7d'] = df['High'] > df['Previous_High_7d']
  df['Higher_Low_7d'] = df['Low'] > df['Previous_Low_7d']

  # Monthly indicators (21 days)
  df['Avg_Volume_21d'] = df['Volume'].shift(1).rolling(window=21).mean()
  df['Relative_Volume_21d'] = df['Volume'] / df['Avg_Volume_21d']

  df['Previous_High_21d'] = df['High'].shift(1).rolling(window=21).max()
  df['Previous_Low_21d'] = df['Low'].shift(1).rolling(window=21).min()
  df['Higher_High_21d'] = df['High'] > df['Previous_High_21d']
  df['Higher_Low_21d'] = df['Low'] > df['Previous_Low_21d']

  # Fill NaNs and Infs safely
  df.replace([np.inf, -np.inf], 0, inplace=True)

  # ملء القيم المفقودة بناءً على نوع العمود
  for col in df.columns:
      if df[col].dtype in ['float64', 'int64']:
         df[col] = df[col].fillna(0)
      elif df[col].dtype == 'bool':
         df[col] = df[col].fillna(0)
      else:
         df[col] = df[col].fillna(0)  # لأي أنواع أخرى (احتياطي)

  return df

In [7]:
egx30 = ['ABUK', 'ADIB', 'ALCN', 'AMOC', 'BTFH', 'COMI', 'EAST', 'EFID', 'HRHO', 'EFIH','EKHO','EKHOA', 'FWRY', 'JUFO', 'GBCO', 'MASR', 'MFPC', 'ORAS', 'ORWE', 'PHDC', 'SKPC', 'SWDY', 'TMGH', 'ETEL', 'EMFD', 'CNFN', 'ISPH', 'ORHD', 'CIEB', 'EGAL', 'RMDA', 'CCAP', 'PHAR']
egx70 = ['UEGC', 'SUGR', 'ASCM', 'CSAG', 'ELEC', 'LCSW', 'OFH', 'ARCC', 'SAUD', 'KRDI', 'ACGC', 'COSG', 'CICH', 'DSCW', 'MPRC', 'KABO', 'KZPC', 'FAITA', 'EHDR', 'ACAMD', 'POUL', 'ELSH', 'PRCL', 'MEPA', 'MCQE', 'MICH', 'MTIE', 'EGCH', 'MPCI', 'ETRS', 'EGAS', 'OLFI', 'OIH', 'EXPA', 'MCRO', 'OCDI', 'MENA', 'ADPC', 'IFAP', 'ATQA', 'PRDC', 'MOIL', 'RACC', 'ATLC', 'ISMQ', 'RAYA', 'BIOC', 'TALM', 'ANFI', 'ZMID', 'AFMC', 'SMFR', 'ENGC', 'ODIN', 'AMER', 'IEEC', 'SVCE', 'AMIA', 'ECAP', 'AJWA', 'EGTS', 'ISMA', 'ZEOT', 'BINV', 'CIRA', 'UBEE', 'CLHO', 'ARAB', 'SCEM', 'ELKA', 'RREI', 'ICFC', 'AFDI', 'SDTI', 'QNBE', 'HELI', 'MPCO']
egx = egx30+egx70


In [11]:
egx_df = pd.DataFrame()
bad_symbols = []
for i in egx:
    tempp_dic = {'Name':'','score':0,'weekly description':'','monthly description':'','Buy or Sell':'','signal power':''}
    try:
        df = pd.read_csv(f'/content/EGX_Datasets/{i}_data.csv', index_col=0)
    except:
        path_df = get_data_tradingView(i)
        # تحميل البيانات
        if path_df is None:
          print(f"Skipping symbol {i} due to data retrieval error.")
          bad_symbols.append(i)
          continue
        else:
          df = pd.read_csv(path_df, index_col=0)


    df = add_indicators(df)
    df.reset_index(inplace=True)
    df.dropna(inplace=True)

    # Apply description functions and unpack the tuple results
    weekly_results = df.apply(lambda row: create_weekly_text_description_ar(row, df), axis=1)
    monthly_results = df.apply(lambda row: create_monthly_text_description_ar(row, df), axis=1)

    # weekly_results لازم يكون list أو Series من tuples (description, score)
    df['weekly description'] = [x[0] for x in weekly_results]
    df['weekly score'] = [x[1] for x in weekly_results]

    df['monthly description'] = [x[0] for x in monthly_results]
    df['monthly score'] = [x[1] for x in monthly_results]


    # Calculate total score
    df['score'] = df['weekly score'] + df['monthly score']
    df.drop(['weekly score', 'monthly score'], axis=1, inplace=True)

    # Populate dictionary with the latest values
    tempp_dic['Name'] = i
    tempp_dic['score'] = df['score'].iloc[-1]
    tempp_dic['weekly description'] = df['weekly description'].iloc[-1]
    tempp_dic['monthly description'] = df['monthly description'].iloc[-1]
    tempp_dic['Buy or Sell'],tempp_dic['signal power'] = check_last_day_signal(df)

    # Append to egx_df
    egx_df = pd.concat([egx_df, pd.DataFrame([tempp_dic])], ignore_index=True)

    col_list = ['High', 'Low', 'Open', 'Close', 'Volume', 'score']
    df = df[col_list]

    try:
      os.mkdir('/content/EGX_Datasets_score')
      df.to_csv(f'/content/EGX_Datasets_score/{i}_data.csv')
    except:
      df.to_csv(f'/content/EGX_Datasets_score/{i}_data.csv')

In [12]:
# التأكد من تحويل الأعمدة لأرقام
egx_df['score'] = pd.to_numeric(egx_df['score'], errors='coerce')
egx_df['signal power'] = pd.to_numeric(egx_df['signal power'], errors='coerce')

# إضافة عمود مؤقت لتحديد الصفوف اللي signal power > 3
egx_df['signal_power_priority'] = egx_df['signal power'] >= 3

# الترتيب بناءً على العمود المؤقت (signal_power_priority) أولاً، وبعدين score
egx_df = egx_df.sort_values(by=['signal_power_priority', 'score'], ascending=[False, False])

# حذف العمود المؤقت
egx_df = egx_df.drop(columns=['signal_power_priority'])

# إعادة ترقيم الـ index
egx_df.reset_index(drop=True, inplace=True)

# عرض الـ DataFrame
egx_df

,Name,score,weekly description,monthly description,Buy or Sell,signal power
0,MCQE,14.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,"شراء (MACD, OBV, Ichimoku)",3
1,ETRS,14.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,"شراء (MACD, SMA, ADX, OBV)",4
2,OFH,13.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,"شراء (MACD, SMA, OBV)",3
3,ATLC,12.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,"شراء (MACD, ADX, OBV)",3
4,HELI,12.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,"شراء (MACD, ADX, Ichimoku)",3
...,...,...,...,...,...,...
105,TALM,-10.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,محايد,0
106,ANFI,-11.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,"بيع (MACD, ADX, OBV)",-3
107,ORWE,-12.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,محايد,0
108,MOIL,-12.0,خلال ال 7 أيام الماضية يمكن وصف حركة السهم كال...,خلال الـ 30 يوم الماضية، يمكن تلخيص أداء السهم...,محايد,0


In [13]:
from google.colab import data_table
data_table.DataTable(egx_df, include_index=False, num_rows_per_page=10)
html_path = "egx_table.html"
egx_df.to_html(html_path, index=False, justify='left')


In [14]:
egx_df.to_csv(f'egx_df{datetime.now()}.csv', index=False)
# egx_df.to_excel(f'egx_df{datetime.now()}.xlsx', index=False)

In [15]:
bad_symbols

[]

In [16]:
!wget -q https://fonts.gstatic.com/ea/notonaskharabic/v3/NotoNaskhArabic-Regular.ttf

In [17]:
html_with_font = '''
<html>
<head>
  <meta charset="utf-8">
  <style>
    @font-face {{
      font-family: 'NotoNaskhArabic';
      src: url('NotoNaskhArabic-Regular.ttf');
    }}
    body {{
      font-family: 'NotoNaskhArabic', sans-serif;
      direction: rtl;
    }}
    table {{
      border-collapse: collapse;
      width: 100%;
    }}
    td, th {{
      border: 1px solid #dddddd;
      padding: 8px;
      text-align: right;
    }}
  </style>
</head>
<body>
  {table}
</body>
</html>
'''
html_content = html_with_font.format(table=egx_df.to_html(index=False, justify='left'))
with open("egx_table_with_font.html", "w", encoding="utf-8") as f:
    f.write(html_content)


In [20]:
!pip -q install weasyprint
from weasyprint import HTML

HTML('egx_table_with_font.html').write_pdf(f'egx_table_{datetime.now().strftime("%Y-%m-%d")}.pdf')

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.003s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.007s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'glyf' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'glyf' tabl

In [26]:
filtered_list = egx_df['Name'].values[:3].tolist()
filtered_list

['MCQE', 'ETRS', 'OFH']

In [ ]:
!git clone https://github.com/zhouhaoyi/Informer2020.git
!cd Informer2020
!pip install -r /content/Informer2020/requirements.txt

In [ ]:
def predict_with_informer(csv_path, des, target):

  command = [
      "python", "-u", "/content/Informer2020/main_informer.py",
      "--model", "informer",
      "--data", "custom",
      "--data_path", csv_path,
      "--features", "S",
      "--target", target,
      "--seq_len", "48",
      "--label_len", "24",
      "--pred_len", "7",
      "--e_layers", "1",
      "--d_layers", "1",
      "--d_model", "256",
      "--n_heads", "8",
      "--d_ff", "2048",
      "--enc_in", "1",
      "--dec_in", "1",
      "--c_out", "1",
      "--freq", "d",
      "--embed", "timeF",
      "--lradj", "type2",
      "--itr", "1",
      "--train_epochs", "20",
      "--batch_size", "32",
      "--patience", "10",
      "--learning_rate", "0.0001",
      "--des", des,
      "--inverse",
      "--do_predict",
  ]

  # تنفيذ الأمر باستخدام subprocess
  process = subprocess.run(command, capture_output=True, text=True)
  print(process.stdout)  # طباعة الـ Output عشان تشوف النتايج
  if process.stderr:
      print("Error:", process.stderr)  # طباعة الأخطاء لو حصلت

In [ ]:
forcasting_df = pd.DataFrame()

for i in filtered_list:
    df = pd.read_csv(f'/content/EGX_Datasets/{i}_data.csv', index_col=0)
    df.dropna(inplace=True)

    features_to_scale = df.columns
    scalers = {col: MinMaxScaler() for col in features_to_scale}
    for col in features_to_scale:
        df[col] = scalers[col].fit_transform(df[[col]])

    df['date'] = df.index
    df['date'] = pd.to_datetime(df['date'])
    df = df[-756:]
    df.reset_index(drop=True, inplace=True)
    df_path = f'/content/{i}_scaled.CA_data.csv'  # غيرت الاسم لتوضيح إنه scaled
    df.to_csv(df_path)

    pred_dic = {'day_1': 0, 'day_2': 0, 'day_3': 0, 'day_4': 0, 'day_5': 0, 'day_6': 0, 'day_7': 0}

    for u in ['High', 'Close', 'Low']:
        des = f'{i} {u}'
        predict_with_informer(df_path, des, u)
        result_path = f'/content/results/informer_custom_ftMS_sl48_ll24_pl7_dm256_nh8_el1_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_{des}_0/real_prediction.npy'

        try:
            future_pred_list = np.load(result_path)
            future_pred_list_2d = future_pred_list[0]  # يصير (7, 1)
            future_pred_list_original = scalers[u].inverse_transform(future_pred_list_2d)
            for j in range(7):
                pred_dic[f'day_{j+1}'] = float(future_pred_list_original[j, 0])
            forcasting_df = pd.concat([forcasting_df, pd.DataFrame(pred_dic, index=[des])])
        except FileNotFoundError:
            print(f"Error: Prediction file for {des} not found at {result_path}.")
            continue

forcasting_df.to_csv('/content/informer_forcasting_df.csv')

In [ ]:
forcasting_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# نفترض انك بالفعل قريت csv
# forcasting_df = pd.read_csv("forcasting_df.csv", header=None)

# عدد الأيام
days = forcasting_df.columns

# عدد الشركات = عدد الصفوف / 3
num_companies = forcasting_df.shape[0] // 3

for i in range(num_companies):
    # استخراج الصفوف الخاصة بالشركة
    high = forcasting_df.iloc[i*3]
    close = forcasting_df.iloc[i*3 + 1]
    low = forcasting_df.iloc[i*3 + 2]

    company_name = forcasting_df.index[i*3][:4]

    plt.figure(figsize=(10, 4))

    plt.plot(days, high, label='High', linestyle='--', marker='^')
    plt.plot(days, close, label='Close', marker='o')
    plt.plot(days, low, label='Low', linestyle='--', marker='v')

    plt.title(f'Forecast for {company_name}')
    plt.xlabel('Future Days')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
